In [1]:
%load_ext autoreload
%autoreload 2
import os; import sys;src_dir = os.path.join(os.getcwd(), os.pardir, 'src');sys.path.append(src_dir)

In [2]:
from data.dal import DAL
import pandas as pd
import numpy as np
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.ensemble
import sklearn.naive_bayes
import xgboost
import tqdm
import sklearn.neighbors
from sklearn.decomposition import PCA
from pandas_ml import ConfusionMatrix
import warnings
warnings.filterwarnings('ignore')

In [3]:
split = DAL("split.hdf")
labels = split.overview()
labels["correct"] = labels.execution_type == 1

In [6]:
featuresdal = DAL("features.hdf")
features = featuresdal.get("tsfresh_raw_min")
features = features[[c for c in features.columns]]
X = features.values

y = labels["execution_type"].values
exercisetrans = sklearn.preprocessing.LabelEncoder()
y = exercisetrans.fit_transform(y)

In [7]:
clf = xgboost.XGBClassifier()
sets = [int(x.replace("skeleton_","")) //100 for x in labels.df_key]
cvstrat = sklearn.model_selection.GroupKFold(10).split(X,groups=sets)
preds = sklearn.model_selection.cross_val_predict(clf,X,y,cv=cvstrat)
a = ["1 None","2 KOT","3 KK","4 FTL"]
ConfusionMatrix(y,preds,a,display_sum=False)

Predicted  1 None  2 KOT  3 KK  4 FTL
Actual                               
1 None        816     30    19     30
2 KOT         161     10    20      5
3 KK          110     26    52      6
4 FTL          40      8     8    449

In [8]:
clf = xgboost.XGBClassifier()
cvstrat = sklearn.model_selection.GroupKFold(10).split(X,groups=labels.person)
preds = sklearn.model_selection.cross_val_predict(clf,X,y,cv=cvstrat)
a = ["1 None","2 KOT","3 KK","4 FTL"]
ConfusionMatrix(y,preds,a,display_sum=False)

Predicted  1 None  2 KOT  3 KK  4 FTL
Actual                               
1 None        724     56    66     49
2 KOT         121     38    28      9
3 KK           75     10   101      8
4 FTL          37      3     6    459

In [5]:


clf = xgboost.XGBClassifier()
cv = sklearn.model_selection.KFold(10,shuffle=True,random_state=0).split(X)
preds = sklearn.model_selection.cross_val_predict(clf,X,y,cv=cv)

In [6]:
a = ["1 None","2 KOT","3 KK","4 FTL"]
#a.reverse()
#ConfusionMatrix([a[i] for i in y],[a[i] for i in preds])
ConfusionMatrix(y,preds,a,display_sum=False)

Predicted  1 None  2 KOT  3 KK  4 FTL
Actual                               
1 None        864      5    11     15
2 KOT          81    102     9      4
3 KK           50      9   134      1
4 FTL          16      0     1    488

Predicted  1 None  2 KOT  3 KK  4 FTL
Actual                               
1 None        724     56    66     49
2 KOT         121     38    28      9
3 KK           75     10   101      8
4 FTL          37      3     6    459